# How to import Ansys meshes
This notebook demonstrates how to import simulation results from Ansys in pylife, e.g., to be used for a FKM nonlinear assessment. This notebook uses the `pymapdl` reader package, so no installation of Ansys is required.
You may need to install `pip install ansys-mapdl-reader` though.

Because working with large meshes can be time-consuming, we also show how a mesh dataset can be filtered.

In [ ]:
# base packages
import os
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import timeit

# Ansys import
from ansys.mapdl import reader as pymapdl_reader

# import pylife.vmap
# pylife
from pylife.strength import failure_probability as fp

import pylife
import pylife.vmap
import pylife.mesh
import pylife.stress.equistress
import pylife.mesh.gradient

Read ANSYS file

In [ ]:
filename = 'data/kt1.rst'
result = pymapdl_reader.read_binary(filename)
print(f"The mesh has {result.mesh.n_node} nodes.")

Process the mesh data and transform it into the pylife mesh format.

In [ ]:
element_number_list = result.element_stress(0)

node_location = result.mesh.nodes
nodal_results = result.nodal_stress(0)
equivalent_results = result.principal_nodal_stress(0)

node_location_df = pd.DataFrame(node_location, columns=['x', 'y', 'z'])
node_number = pd.DataFrame(nodal_results[0], columns=['node_id'], dtype=int)
nodal_stress = pd.DataFrame(nodal_results[1], columns = ['S11', 'S22', 'S33', 'S12', 'S13', 'S23'])
nodal_equivalent_stress = pd.DataFrame(equivalent_results[1], columns = ['S1', 'S2', 'S3', 'S_Int', 'mises'])

data = [node_number, node_location_df, nodal_stress, nodal_equivalent_stress]

mesh_data = pd.concat(data, axis=1)
mesh_data = mesh_data.set_index("node_id")

# create multi-index for FE mesh in pylife
tuples = [(elem[8], elem[10+i]) for elem in result.mesh.elem for i in range(len(elem)-10)]
index = pd.MultiIndex.from_tuples(tuples, 
                          names=["element_id", "node_id"])
# sort multi-index by element_id, but keep order of node-ids per element
index,_ = index.sortlevel(0, sort_remaining=False)   

# join mesh data with multi index
pylife_mesh = pd.DataFrame(index=index)
pylife_mesh = pylife_mesh.join(mesh_data, on="node_id")

Now, we have the mesh as a pandas DataFrame with multi-index:

In [ ]:
pylife_mesh

In [ ]:
# Calculate the stress gradient
tstart = timeit.default_timer()
grad1 = pylife_mesh.gradient_3D.gradient_of('mises')
tend = timeit.default_timer()
print(f"duration calculate stress gradient: {tend-tstart:.1f} s")

grad1["abs_grad"] = np.linalg.norm(grad1, axis=1)
pylife_mesh = pylife_mesh.join(grad1, sort=False)

## Plot the mesh
Plot the absolute stress gradient G.

In [ ]:
import pyvista as pv

# import profile
mpl.style.use('bmh')

grid = pv.UnstructuredGrid(*pylife_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=pylife_mesh.groupby('element_id')['abs_grad'].mean().to_numpy(),
                show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()
# =============================================================================

Plot the Mises stress.

In [ ]:
grid = pv.UnstructuredGrid(*pylife_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=pylife_mesh.groupby('element_id')['mises'].mean().to_numpy(),
                show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()

## Select surface elements
Often, we want to only consider the finite elements that touch the surface of the 3D object, because the highest fatigue load is often located at the surface.
A heuristic is to count the number of adjacent elements for each node and filter the node set by this number. At the surface, less elements are next to each other than in the interior of the volume. The actual number depends on the used element types.

In [ ]:
# compute number of elements that are adjacent for each node
pylife_mesh_temp = pylife_mesh.copy()
pylife_mesh_temp["element_id"] = pylife_mesh_temp.index.get_level_values("element_id")
pylife_mesh_temp = pylife_mesh_temp.droplevel("element_id")
n_elements_per_node = pylife_mesh_temp.groupby("node_id")["element_id"].nunique()


In [ ]:
# How many elements max. should touch at a node to be considered a surface node?
n_elements = 2  # for a hexahedral mesh
#n_elements = 4  # for a tetrahedral mesh

# select nodes with less than a certain number of elements
surface_node_ids = n_elements_per_node[n_elements_per_node<=n_elements].index
surface_element_ids = pylife_mesh_temp.loc[surface_node_ids,"element_id"].unique()
surface_mesh = pylife_mesh[pylife_mesh.index.get_level_values("element_id").isin(surface_element_ids)]

n_elements_original = pylife_mesh.index.get_level_values("element_id").nunique()
print(f"select {len(surface_element_ids)} elements of {n_elements_original} "
      f"(= {100*len(surface_element_ids)/n_elements_original:.1f} %)")

The resulting mesh at the surface has the following elements:

In [ ]:
surface_mesh

The following visualization shows only the mesh with only surface elements. It looks the same as the full mesh, though.

In [ ]:
grid = pv.UnstructuredGrid(*surface_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=surface_mesh.groupby('element_id')['mises'].mean().to_numpy(),
                show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()

## Select nodes with highest stress
We can also filter the finite element mesh by the Mises stress, only considering nodes with a stress above a certain threshold.

In [ ]:
# The threshold is specified in relation to the maximum stress, by the following factor:
stress_factor = 0.8

# select elements where at least one node has at least the given mises stress
pylife_mesh_temp = pylife_mesh.copy()
maximum_stress = pylife_mesh_temp["mises"].max()
selected_node_ids = pylife_mesh_temp[pylife_mesh_temp["mises"] > stress_factor*maximum_stress].index.get_level_values("node_id")
selected_element_ids = pylife_mesh_temp[pylife_mesh_temp.index.get_level_values("node_id").isin(selected_node_ids)].index.get_level_values("element_id").unique()
selected_mesh = pylife_mesh[pylife_mesh.index.get_level_values("element_id").isin(selected_element_ids)]

# output statistic
n_elements_original = pylife_mesh.index.get_level_values("element_id").nunique()
print(f"select {len(selected_element_ids)} elements of {n_elements_original} "
      f"(= {100*len(selected_element_ids)/n_elements_original:.1f} %)")

In [ ]:
grid = pv.UnstructuredGrid(*selected_mesh.mesh.vtk_data())
plotter = pv.Plotter(window_size=[1920, 1080])
plotter.add_mesh(grid, scalars=selected_mesh.groupby('element_id')['mises'].mean().to_numpy(),
                show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()